### Move Joy Flows to MongoDB

This reads a set of JSON files and moves them to the MongoDB for faster post processing.

The JSON file is created with "./bin/joy bidir=1 dist=1 <pcap_file> > <file>.json
    
Note:  Make sure Joy has been configured to be compiled NOT to generate GZIP output
-- ./configure    # do NOT include --enable_gzip

In [1]:
import sys, getopt
import time
from pymongo import MongoClient
import json

In [2]:
def initMongodb(uri):
    try:
        client = MongoClient(uri)
        try:
            client.drop_database('flowdb')
        except:
            pass
        # Create the new empty DB
        db = client.flowdb
    except Exception as e:
        print(e)
        sys.exit(1)
        
    return db

In [3]:
# JSON Files to move to MongoDB

# Training Data
p_train = "./piracy_train/piracy.json"
b_train = "./benign_train/benign.json"

# Test Data
b_test = "./test_data/benign1.json"
p_test = "./test_data/iptvshop.json"

test_data = "/media/sf_TestDataSet/mediacom1.json"

db_name = "flowdb"
p_train_coll = "p2"
p_test_coll = "testP"
b_train_coll = "b2"
b_test_coll = "testB"
net_train_coll = "Netflow"
ab_test_coll = "bWeb"
FullNet_c = 'mediacom2'

In [4]:
# Read the file line by line and write MongoDB
def process_file(filename, db, collection):
    flowsadded = 0
    with open(filename,'r') as file:
        for line in file:
            if 'version' not in line:
                aline = ''.join(char for char in line if ord(char) != 45)
                j = json.loads(aline)
                try:
                    dbResult = db[collection].insert_one(j)
                    flowsadded = flowsadded + 1
                except Exception as e:
                    print(e)
    print "Added:", flowsadded

In [5]:

#mongo_uri = atlas_connection
mongo_uri = "mongodb://localhost:27017/"

db = initMongodb(mongo_uri)

# Now move each JSON file to the MongoDB
process_file(p_train, db, p_train_coll)
process_file(b_train, db, b_train_coll)
process_file(b_test, db, b_test_coll)
process_file(p_test, db, p_test_coll)
process_file(b_test, db, ab_test_coll)
process_file(test_data, db, FullNet_c)

print "Done"
#print db.flowdb.stats()

Added: 44271
Added: 34156
Added: 3798
Added: 2737
Added: 3798


KeyboardInterrupt: 